In [24]:
!pip install vk_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 721.4 kB/s eta 0:00:000:00:01


In [25]:
#import needed moduls 
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json 
import time
import vk_api

<h2>API VK

In [207]:
api_version = '5.199'      # lastes api version
client_id = '####' # client_id of my app

scope = 'friends,wall,status,wall'                  # scope parameters
print('Your API VK generation link: \n')
print(f'https://oauth.vk.com/authorize?client_id={client_id}&display=page&scope={scope}&redirect_uri=https://oauth.vk.com/blank.html&response_type=token&v={api_version}&state=123456')


Your API VK generation link: 

https://oauth.vk.com/authorize?client_id=####&display=page&scope=friends,wall,status,wall&redirect_uri=https://oauth.vk.com/blank.html&response_type=token&v=5.199&state=123456


In [20]:
token = '####'

In [177]:
links = ['https://vk.com/public184752831', 'https://vk.com/podslyhano_yasenevo', 'https://vk.com/yasenevo_konkovo_tepliistan']

<h2>Commentaries parsing function</h2>


In [191]:
def get_id(link):
    '''
    This functions transform vk link to id
    
    Arguments:
        > link - str - a full vk link
    Returns:
        > id - str - a group id
    '''
    name = link.split('/')[-1]
    id = "-" + str(vk.groups.getById(group_id = name, fields = 'id')[0]['id']) #use "-" only for groups
    return id
def parse_comments(owner_id = '-184752831', post_id = '2036'):
    '''
    This fuction using VK API to parse a text of comments in community from link and returns a pandas.DataFrame

    Arguments:
        > owner_id - str -  an id of group
        > post_id - str - a post id
    Returns:
        >  comments - pandas.DataFrame - with users id and comment text:
            : user_id - str - id
            : comment - str - text comment 
    '''
    comments_counter = vk.wall.getComments(owner_id = owner_id, post_id = post_id)['count']
    comments = pd.DataFrame()
    temp_df = pd.DataFrame()
    comments_data = vk.wall.getComments(owner_id=owner_id, post_id=post_id)['items']
    if comments_counter > 0:
        for i in range(min(comments_counter, len(comments_data))):
            temp_df = pd.json_normalize(vk.wall.getComments(owner_id = owner_id, post_id = post_id)['items'][i])
            comments = pd.concat([temp_df, comments])
    return comments
    
def get_posts(link, count = 10):
    '''
    This function get group vk link and return all comments from first 10 (by default) posts 

    Arguments:
        > link - str - full link to group
        > count - int - count of first posts (10, by default)
    '''
    owner_id = get_id(link)
    posts = vk.wall.get(owner_id=owner_id, count=count)['items']
    post_ids = [i['id'] for i in posts]
    answer = pd.DataFrame()
    for i in post_ids:
        tm_answer = parse_comments(owner_id = owner_id, post_id = i)
        answer = pd.concat([tm_answer,answer])
        time.sleep(0.5)
    return answer


<h3>Let's try at links</h3>

In [180]:
full_df = pd.DataFrame()
for link in links:
    temp_df = get_posts(link)
    full_df = pd.concat([temp_df, full_df])

full_df = full_df[['from_id', 'text']]

In [181]:
full_df        time.wait(0.5)

,from_id,text
0,785108709,Логично кассиры нужны обязательно везде на каж...
0,584561742,а это кому-то нужно? лучше бы эти дебилы касси...
0,584561742,за биометрию как беспонтовое расходование сред...
0,51541780,А где предупреждение их не трогать🤔
0,31873820,Надо перевести на карту 10 руб и написать сооб...
0,411094904,"Скора 1 поездка будет 500 рэ,а зарплата та же 👍"
0,584561742,могли бы уж сразу 60 сделать для ровного счёта...
0,31873820,Красивый Зверь.
0,203637850,"Да, могу для вас снять пару сцен"
0,8872382,"Здравствуйте. У нас ""на районе"" целый кинологи..."


In [192]:
df = get_posts('https://vk.com/zloy_ekaterinburg', count = 150)

In [205]:
df

,id,from_id,date,text,post_id,owner_id,parents_stack,thread.count,thread.items,thread.can_post,thread.show_reply_button,attachments,thread.next_from
0,341,546115349,1684346713,"Это я так хотела обратиться,чтобы кровать маль...",192,-172482613,[],0,[],True,True,NaN,NaN
0,241,236358939,1648336773,"Ну, ребята, сначала нужно узнать стоимость раб...",192,-172482613,[],0,[],True,True,NaN,NaN
0,225,132825808,1645002119,Яб косарь кинул и выпнул черта,192,-172482613,[],0,[],True,True,NaN,NaN
0,198,43633150,1642248679,А вы заплатили?,192,-172482613,[],0,[],True,True,NaN,NaN
0,197,43633150,1642248340,,192,-172482613,[],0,[],True,True,"[{'type': 'sticker', 'sticker': {'inner_type':...",NaN
0,195,3495858,1642159150,от компухтерных мастеров тоже не отстают,192,-172482613,[],0,[],True,True,NaN,NaN
0,226,11193663,1645243237,"Заберите на растопку, их срубили уже.",196,-172482613,[],0,[],True,True,NaN,NaN
0,203,43633150,1642350292,?,202,-172482613,[],0,[],True,True,NaN,NaN
0,292,764071603,1669492504,Ну он же на Жигулях! Ему можно😁,206,-172482613,[],0,[],True,True,NaN,NaN
0,240,47825211,1648128769,Номер и лицо зачем закрасили? Страна должна зн...,206,-172482613,[],0,[],True,True,NaN,NaN


In [204]:
len(df['from_id']) - len(df['from_id'].unique())

13